# Calibration
## 1 Setting up a calibrator using configuration saved on file
## 2 Running a calibration
## 3 Changing calibration settings
### 3.1 Calibration period (Time-axis)
### 3.2 Parameter ranges
### 3.3 Initial state
### 3.3.1 Setting spatially invariable state
### 3.3.2 Setting state using a warm-up period
### 3.4 Target series
#### 3.4.1 Activating/deactivating target series
#### 3.4.2 Changing time-axis (period and/or resolution)
## 4 Run a simulation using calibrated parameters
### 4.1 Activating cells in  subcatchments not represented by the target series
### 4.2 Running using a light-weight model
### 4.3 Running using a full model

In [ ]:
import os
from shyft import api
from shyft.orchestration.configuration.yaml_configs import YAMLCalibConfig
from shyft.orchestration.simulators.config_simulator import ConfigCalibrator

In [ ]:
# 1.Setting up a calibrator using configuration saved on file
config_dir = "D:/users/ysa/shyft_config/yaml"
config_file = os.path.join(config_dir, "calibration_config.yaml")
region_name = "Nea-Nidelv"
cfg = YAMLCalibConfig(config_file, region_name)
calibrator = ConfigCalibrator(cfg)

In [ ]:
# 2.Running a calibration
region_model_id = calibrator.region_model_id
interpolation_id = calibrator.interpolation_id

region_model_repo = calibrator.region_model_repository
interpolation_param_repo = calibrator.ip_repos
geo_ts_repo = calibrator.geo_ts_repository
initial_state_repo = calibrator.initial_state_repo

geo_ts_names = ("temperature", "wind_speed", "precipitation", "relative_humidity", "radiation")

#region_model = region_model_repo.get_region_model(region_model_id)
region_model = calibrator.region_model
optimizer = calibrator.optimizer

optim_method_name = calibrator.optim_method
optim_method_params = calibrator.optim_method_params
param_init = calibrator.p_init

optimization_method = {'min_bobyqa': optimizer.optimize,
                       'dream': optimizer.optimize_dream,
                       'sceua': optimizer.optimize_sceua}

epsg = region_model.bounding_region.epsg()
bbox = region_model.bounding_region.bounding_box(epsg)
period = region_model.time_axis.total_period()

sources = geo_ts_repo.get_timeseries(geo_ts_names, period, geo_location_criteria=bbox)


def get_region_env(sources_):
    region_env_ = api.ARegionEnvironment()
    region_env_.temperature = sources_["temperature"]
    region_env_.precipitation = sources_["precipitation"]
    region_env_.radiation = sources_["radiation"]
    region_env_.wind_speed = sources_["wind_speed"]
    region_env_.rel_hum = sources_["relative_humidity"]
    return region_env_

interpolation_parameters = interpolation_param_repo.get_parameters(interpolation_id)
region_env = get_region_env(sources)
#region_model.run_interpolation(interpolation_parameters, region_model.time_axis, region_env)
region_model.interpolate(interpolation_parameters, region_env)


def get_init_state_from_repo(initial_state_repo_, region_model_id_=None, timestamp=None):
    state_id = 0
    if hasattr(initial_state_repo_, 'n'):  # No stored state, generated on-the-fly
        initial_state_repo_.n = region_model.size()
    else:
        states = initial_state_repo_.find_state(
            region_model_id_criteria=region_model_id_,
            utc_timestamp_criteria=timestamp)
        if len(states) > 0:
            state_id = states[0].state_id  # most_recent_state i.e. <= start time
        else:
            raise Exception('No initial state matching criteria.')
    return initial_state_repo_.get_state(state_id)

init_state = get_init_state_from_repo(initial_state_repo, region_model_id_=region_model_id, timestamp=region_model.time_axis.start)
region_model.initial_state = init_state

param_init_vct = [param_init.get(i) for i in range(param_init.size())]
optim_param_vct = optimization_method[optim_method_name](param_init_vct, **optim_method_params)
optim_param = region_model.parameter_t()
optim_param.set(optim_param_vct)

In [ ]:
# 2.Changing calibration settings
# 2.1Calibration period (Time-axis)
ta_orig = region_model.time_axis
ta_new = api.Timeaxis(ta_orig.start-ta_orig.delta_t*ta_orig.n, ta_orig.delta_t, ta_orig.n)

period = ta_new.total_period()
sources = geo_ts_repo.get_timeseries(geo_ts_names, period, geo_location_criteria=bbox)
region_env = get_region_env(sources)
region_model.run_interpolation(interpolation_parameters, ta_new, region_env)
init_state = get_init_state_from_repo(initial_state_repo, region_model_id_=region_model_id, timestamp=region_model.time_axis.start)
region_model.initial_state = init_state

target_spec_vct = calibrator._create_target_specvect()
optimizer.target_specification = target_spec_vct

In [ ]:
# 2.2Parameter ranges
param_min_orig = region_model.parameter_t(optimizer.parameter_lower_bound)
param_max_orig = region_model.parameter_t(optimizer.parameter_upper_bound)
optimizer.parameter_lower_bound.gs.tx -= 1.
optimizer.parameter_upper_bound.gs.tx += 1.

In [ ]:
# 2.3Initial state
# 2.3.1Setting spatially invariable state
from shyft.repository.generated_state_repository import GeneratedStateRepository
init_values = {'gamma_snow':
                   {'acc_melt': 0.0,
                    'albedo': 0.65,
                    'alpha': 6.25,
                    'iso_pot_energy': 0.0,
                    'lwc': 0.1,
                    'sdc_melt_mean': 0.0,
                    'surface_heat': 30000.0,
                    'temp_swe': 0.0},
              'kirchner':
                  {'q': 0.01}
              }
model_type = region_model.__class__
uniform_initial_state_repo = GeneratedStateRepository(model_type, init_values)
init_state_uniform = get_init_state_from_repo(uniform_initial_state_repo)
region_model.initial_state = init_state_uniform

In [ ]:
# 2.3.2Setting state using a warm-up period
ta_main_sim = region_model.time_axis
cal = api.Calendar()
warm_up_period = cal.YEAR
ta_warm_up = api.Timeaxis(ta_main_sim.start-cal.YEAR, ta_main_sim.delta_t, int(cal.YEAR/ta_main_sim.delta_t))
region_model_warm_up = region_model.__class__(region_model.extract_geo_cell_data(), region_model.get_region_parameter())
region_model_warm_up.initialize_cell_environment(ta_warm_up)
period = region_model_warm_up.time_axis.total_period()
sources = geo_ts_repo.get_timeseries(geo_ts_names, period, geo_location_criteria=bbox)
region_env = get_region_env(sources)
region_model_warm_up.interpolate(interpolation_parameters, region_env)
init_state_for_warm_up = get_init_state_from_repo(uniform_initial_state_repo)
region_model_warm_up.set_states(init_state_for_warm_up)
region_model_warm_up.run_cells()
init_state_from_warm_up = region_model_warm_up.__class__.state_t.vector_t()
region_model_warm_up.get_states(init_state_from_warm_up)
region_model.initial_state = init_state_from_warm_up

In [ ]:
# 2.4Target series
# 2.4.1Activating/deactivating target series
target_spec_vct_new = api.TargetSpecificationVector()
target_spec_vct_orig = optimizer.target_specification
target_names_to_exclude = [target_spec_vct_orig[0].uid]
[target_spec_vct_new.extend(target) for target in target_spec_vct_orig if target.uid not in target_names_to_exclude]
optimizer.target_specification = target_spec_vct_new

In [ ]:
# 2.4.2Changing time-axis

In [ ]:
# 3.Run a simulation using calibrated parameters
# 3.1Activating cells in  subcatchments not represented by the target series

In [ ]:
# 3.2Running using a light-weight model

In [ ]:
# 3.3Running using a full model